In [1]:
%load_ext autoreload
%autoreload 2
from data_api import *

In [2]:
database = database_api('/Users/ziandong/TreeMusicRecommendation/Data/conf.json')
last_fm = lastfm_api('/Users/ziandong/TreeMusicRecommendation/Data/conf.json')
result = last_fm.get_user_friends(username='rj')

In [5]:
"construrt userdatabse with 100000 users, Do not run it again"
import queue
start_username = "Kimgs"
user_queue = queue.Queue()
user_queue.put(start_username)
users_processed = set()
users_processed.add(start_username)

while not user_queue.empty() and len(users_processed) < 10000:
    current_username = user_queue.get()
    friends = last_fm.get_user_friends(current_username)

    if friends:
        for friend in friends:
            friend_name = friend['name']
            if friend_name not in users_processed:
                users_processed.add(friend_name)
                user_queue.put(friend_name)
                database.save_users([friend])  # Assuming save_users takes a list
                
                # If we've reached 10,000 users, we can break early
                if len(users_processed) >= 10000:
                    break

print(f"Total users in database: {len(users_processed)}")


Total users in database: 10000


In [6]:
# 备份 user 表
# cursor = database.cnx_cursor
# source_table = 'Users'
# new_table = 'Users_copy'

# drop the table if it exists
# drop_table_query = f"DROP TABLE IF EXISTS {new_table};"
# cursor.execute(drop_table_query)

# SQL statement to create a new table as a copy of another
# create_table_query = f"CREATE TABLE {new_table} LIKE {source_table};"
# cursor.execute(create_table_query)

# SQL statement to copy all data from the source table to the new table
# insert_data_query = f"INSERT INTO {new_table} SELECT * FROM {source_table};"
# cursor.execute(insert_data_query)

# Commit the changes
# database.cnx.commit()


In [3]:
cursor = database.cnx_cursor
source_table = 'Users_copy'
new_table = 'Users'

# SQL statement to copy all data from the source table to the new table
insert_data_query = f"INSERT INTO {new_table} SELECT * FROM {source_table};"
cursor.execute(insert_data_query)

# Commit the changes
database.cnx.commit()


In [4]:
# construct top_tracks
users = database.get_all_users()
for j in range(0, len(users)):
    try:
        user_id = users[j][0]
        user_name = users[j][1]
        top_tracks = last_fm.get_top_tracks(username=user_name)
        artist_infos = []
        if not top_tracks:
            print(j)
            continue
        for i in range(len(top_tracks)):
                track = top_tracks[i]
                url = track['track_url']
                start = len("https://www.last.fm/music/")
                end = url.index("/_/")
                artist_name = url[start:end]

                top_tracks[i]['artist_name'] =  artist_name
                artist_url = "https://www.last.fm/music/" + artist_name
                artist_info = {"artist_name": artist_name, "artist_url": artist_url}
                artist_infos.append(artist_info)
                top_tracks[i]['track_id'] = track['artist_name'] + ": " + track['track_name']

        
        database.save_artists(artist_infos)
        database.save_tracks(top_tracks)
        database.save_top_tracks(user_id, top_tracks)
    except:
        print('write user '+str(j)+' failed')
        continue


write user3failed
write user18failed
write user20failed
write user32failed
write user36failed
write user41failed
write user50failed
write user53failed
write user60failed
write user68failed
write user73failed
write user74failed
write user76failed
write user78failed
write user94failed
write user103failed
write user115failed
write user126failed
write user133failed
write user134failed
write user135failed
write user143failed
write user145failed
write user149failed
write user171failed
write user180failed
write user181failed
write user190failed
write user206failed
write user207failed
write user218failed
write user234failed
write user235failed
write user237failed
write user249failed
write user273failed
write user315failed
write user326failed
331
write user343failed
write user344failed
write user356failed
write user361failed
write user371failed
write user372failed
write user379failed
write user382failed
write user394failed
write user396failed
write user397failed
write user401failed
write user40

In [12]:
# Construct listening history
users = database.get_all_users()
for j in range(len(users)):
    user_id = users[j][0]
    user_name = users[j][1]
    recent_tracks = last_fm.get_recent_tracks(username=user_name)
    if not recent_tracks:
        continue
    for i in range(len(recent_tracks)):
        try:
            track = recent_tracks[i]
            url = track['track_url']
            start = len("https://www.last.fm/music/")
            end = url.index("/_/")
            artist_name = url[start:end]

            recent_tracks[i]['artist_name'] =  artist_name
            artist_url = "https://www.last.fm/music/" + artist_name
            artist_info = {"artist_name": artist_name, "artist_url": artist_url}
            artist_infos.append(artist_info)
            recent_tracks[i]['track_id'] = track['artist_name'] + ": " + track['track_name']

        except:
            continue

    database.save_artists(artist_infos)
    database.save_tracks(recent_tracks)
    database.save_listening_history(1, recent_tracks)
    print(j)

TypeError: tuple indices must be integers or slices, not str

In [5]:
# copy top_tracks to top_tracks_copy
cursor = database.cnx_cursor
source_table = 'Top_track'
new_table = 'Top_track_copy'

# drop the table if it exists
drop_table_query = f"DROP TABLE IF EXISTS {new_table};"
cursor.execute(drop_table_query)

# SQL statement to create a new table as a copy of another
create_table_query = f"CREATE TABLE {new_table} LIKE {source_table};"
cursor.execute(create_table_query)

# SQL statement to copy all data from the source table to the new table
insert_data_query = f"INSERT INTO {new_table} SELECT * FROM {source_table};"
cursor.execute(insert_data_query)


In [21]:
# test
user_id = 1
user_name = "rj"

top_artists = last_fm.get_top_artist(username=user_name)


# save top artists
database.save_artists(top_artists)
database.save_top_artists(user_id, top_artists)

In [23]:
# save top artists
users = database.get_all_users()

for j in range(len(users)):
    try:
        user_id = users[j][0]
        user_name = users[j][1]
        top_artists = last_fm.get_top_artist(username=user_name)
        database.save_artists(top_artists)
        database.save_top_artists(user_id, top_artists)
        print("insert user "+str(j)+" success")
    except:
        print("insert user "+str(j)+" failed")
        continue


insert user 0 success
insert user 1 success
insert user 2 success
insert user 3 success
insert user 4 success
insert user 5 success
insert user 6 success
insert user 7 success
insert user 8 success
insert user 9 success
insert user 10 success
insert user 11 success
insert user 12 success
insert user 13 success
insert user 14 success
insert user 15 success
insert user 16 success
insert user 17 success
insert user 18 success
insert user 19 success
insert user 20 success
insert user 21 success
insert user 22 success
insert user 23 success
insert user 24 success
insert user 25 success
insert user 26 success
insert user 27 success
insert user 28 success
insert user 29 success
insert user 30 success
insert user 31 success
insert user 32 success
insert user 33 success
insert user 34 success
insert user 35 success
insert user 36 success
insert user 37 success
insert user 38 success
insert user 39 success
insert user 40 success
insert user 41 success
insert user 42 success
insert user 43 succes

In [ ]:
# construct top artists
